In [ ]:
# Instalação de dependências
!pip install transformers datasets scikit-learn

In [ ]:
# Importação e Preparação do Dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar o arquivo JSON
from google.colab import files
uploaded = files.upload()

# Carregar o arquivo JSON
with open("trn.json", "r") as f:
    data = json.load(f)

# Transformar em DataFrame
df = pd.DataFrame(data)

# Manter apenas colunas relevantes
df = df[["title", "content"]].dropna()

# Formatar os prompts para o fine-tuning
df["prompt"] = "Pergunta: Descreva o produto | Contexto: " + df["title"]
df["response"] = df["content"]

# Dividir em treino e validação
train_data, val_data = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# Tokenização e Configuração do Modelo
from transformers import AutoTokenizer, AutoModelForCausalLM

# Escolher o modelo base
model_name = "gpt2"  # Modelo base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenizar os dados
def tokenize_function(example):
    return tokenizer(
        example["prompt"],
        text_target=example["response"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Tokenizar o conjunto de dados
train_encodings = train_data.apply(lambda x: tokenize_function(x), axis=1)
val_encodings = val_data.apply(lambda x: tokenize_function(x), axis=1)

In [ ]:
# Configuração e Execução do Treinamento
from transformers import Trainer, TrainingArguments

# Configurar argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
)

# Preparar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
)

# Fine-tuning
trainer.train()

In [ ]:
# Avaliação e Geração de Respostas
def generate_response(question, title):
    input_text = f"Pergunta: {question} | Contexto: {title}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Teste do modelo treinado
question = "Qual é a descrição do produto?"
title = "Laptop Dell Inspiron 15"
print(generate_response(question, title))